In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
import random
from PIL import Image
from skimage.filters import unsharp_mask

num_of_centroids=5000
num_train_samples=600
num_test_samples=int(0.2*num_train_samples)
K = 11
all_image_idx=random.sample(range(0,1000),num_train_samples+num_test_samples)

In [ ]:
def remove_noise (image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    X = cv2.bilateralFilter(gray, 15, sigmaColor=10, sigmaSpace=10)
    median = cv2.medianBlur(X, 5)
    result_2 = unsharp_mask(median, radius=10, amount=4)*255
    result_2 = np.uint8(result_2)
    sharpen = cv2.Canny(result_2, 100,250)
    return sharpen

In [ ]:
all_des=np.empty((1,128))

sift = cv2.SIFT_create(400)

num_of_desc=[]

for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/IBM Plex Sans Arabic/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 1 image"+str(i))

num_of_desc+=[all_des.shape[0]]
print("FINISHED READING FIRST SET OF IMAGES")

for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/Lemonada/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 2 image"+str(i))

num_of_desc+=[all_des.shape[0]]
print("FINISHED READING SECOND SET OF IMAGES")

for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/Marhey/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 3 image"+str(i))

num_of_desc+=[all_des.shape[0]]
print("FINISHED READING THIRD SET OF IMAGES")

for i in range (num_train_samples):
    image = cv2.imread("../../fonts-dataset/Scheherazade New/"+str(all_image_idx[i])+".jpeg")
    removed_noise = remove_noise(image)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        all_des=np.vstack((all_des,descriptors))
    print("processing type 4 image"+str(i))

num_of_desc+=[all_des.shape[0]]
print("FINISHED READING FOURTH SET OF IMAGES")

desc_labels=np.zeros(all_des.shape[0])
desc_labels[num_of_desc[0]:num_of_desc[1]]=1
desc_labels[num_of_desc[1]:num_of_desc[2]]=2
desc_labels[num_of_desc[2]:num_of_desc[3]]=3

In [ ]:
# different options for kmeans
# - mini batches
# - limiting the number of descriptors
kmeans=MiniBatchKMeans(n_clusters=num_of_centroids,batch_size=num_of_centroids,max_iter=20).fit(X=all_des)

In [ ]:
# for each centroid, calculate how it is near to each label
centroids_labels=np.zeros((num_of_centroids,4))
centroids_labels_SVM=np.zeros((num_of_centroids))

for i in range(num_of_centroids):
    centroids_labels[i][0]=(np.sum(desc_labels[kmeans.labels_==i]==0))  ## kmeans.labels_ -> array to map each descriptor to a centroid
    centroids_labels[i][1]=(np.sum(desc_labels[kmeans.labels_==i]==1))
    centroids_labels[i][2]=(np.sum(desc_labels[kmeans.labels_==i]==2))
    centroids_labels[i][3]=(np.sum(desc_labels[kmeans.labels_==i]==3))
    centroids_labels[i]/=np.sum(centroids_labels[i])
# for each centroid, calculate its label, its label is the label that has the highest sum
    centroids_labels_SVM[i]=np.argmax(centroids_labels[i])

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='poly', C=0.1, random_state=0, coef0=1, degree=4, gamma=10.0,class_weight= None)
svm.fit(kmeans.cluster_centers_, centroids_labels_SVM)

In [ ]:
def predict_svm(path,svm):
    image=cv2.imread(path)
    removed_noise = remove_noise(image)
    _ , descriptors= sift.detectAndCompute(removed_noise,None)

    predictions = np.zeros(4)
    if descriptors is not None:
        for des in descriptors:
            ind = np.int64(svm.predict([des]))
            predictions[ind]+=1
        return np.argmax(predictions)
    else:
        return -1

num_right0=0
num_right1=0
num_right2=0
num_right3=0

for i in range (num_test_samples):
    predic = predict_svm("../../fonts-dataset/IBM Plex Sans Arabic/"+str(all_image_idx[i+num_train_samples])+".jpeg", svm)
    num_right0 += predic==0
    print("processing type 0 image"+str(all_image_idx[i+num_train_samples])+ " prediction: "+str(predic))

print("FINISHED TESTING FIRST SET OF IMAGES")

for i in range (num_test_samples):
    predic = predict_svm("../../fonts-dataset/Lemonada/"+str(all_image_idx[i+num_train_samples])+".jpeg", svm)
    num_right0 += predic==1
    print("processing type 1 image"+str(all_image_idx[i+num_train_samples])+ " prediction: "+str(predic))

print("FINISHED TESTING SECOND SET OF IMAGES")

for i in range (num_test_samples):
    predic = predict_svm("../../fonts-dataset/Marhey/"+str(all_image_idx[i+num_train_samples])+".jpeg", svm)
    num_right0 += predic==2
    print("processing type 2 image"+str(all_image_idx[i+num_train_samples])+ " prediction: "+str(predic))

print("FINISHED TESTING THIRD SET OF IMAGES")

for i in range (num_test_samples):
    predic = predict_svm("../../fonts-dataset/Scheherazade New/"+str(all_image_idx[i+num_train_samples])+".jpeg", svm)
    num_right0 += predic==3
    print("processing type 3 image"+str(all_image_idx[i+num_train_samples])+ " prediction: "+str(predic))

print("FINISHED TESTING FOURTH SET OF IMAGES")
print(((num_right0+num_right1+num_right2+num_right3)/(4*num_test_samples))*100)